In [1]:
import aiohttp
import asyncio
import aiofiles
from bs4 import BeautifulSoup
import pandas as pd
import time
from urllib.parse import urljoin, urlparse
from tqdm import tqdm
import re
import nest_asyncio

In [2]:
# Fix async issues
nest_asyncio.apply()

In [3]:
class AsyncWebScraper:
    def __init__(self, start_url):
        self.start_url = start_url
        self.visited = set()
        self.to_visit = [start_url]
        self.data = []
        self.headers = {"User-Agent": "Mozilla/5.0"}

    async def fetch(self, session, url):
        """Fetch webpage async, but only return HTML content."""
        try:
            async with session.get(url, headers=self.headers, timeout=10) as response:
                if response.status == 200:
                    content_type = response.headers.get("Content-Type", "")
                    if "text/html" in content_type:
                        return await response.text()
                    else:
                        print(f"Skipping non-HTML content: {url}")
                else:
                    print(f"Failed to fetch {url} with status {response.status}")
        except Exception as e:
            print(f"Failed to scrape {url}: {e}")
        return None

    def clean_text(self, text):
        """Improve readability"""
        text = text.strip()
        text = re.sub(r'\s*\|\s*', '\n', text)  # Split on " | " into new lines
        text = re.sub(r'(?<=[a-zα-ω])\s+(?=[A-ZΑ-Ω])', '\n', text)  # Add newline before capitalized words
        text = re.sub(r'(?<=\.)\s+', '\n', text)  # Add new line after a period
        text = re.sub(r'\s{2,}', '\n', text)  # Remove excess spaces
        return text.strip()

    async def scrape_page(self, session, url):
        """Scrape text from a page."""
        if url in self.visited:
            return
        self.visited.add(url)

        print(f"Scraping: {url}")
        html = await self.fetch(session, url)
        if not html:
            return

        soup = BeautifulSoup(html, "html.parser")
        for element in soup(["script", "style", "noscript"]):
            element.extract()

        # Extract and clean text
        texts = [self.clean_text(text) for text in soup.stripped_strings if len(text) > 3]
        self.data.append({"url": url, "text": "\n".join(texts)})

        # Extract internal links and add them to queue, skipping common image files
        for link in soup.find_all("a", href=True):
            absolute_url = urljoin(url, link["href"])
            if (self.is_internal_url(absolute_url) and 
                absolute_url not in self.visited and 
                not re.search(r"\.(jpg|jpeg|png|gif|svg)$", absolute_url, re.IGNORECASE)):
                self.to_visit.append(absolute_url)

    def is_internal_url(self, url):
        """Check if a URL belongs to the same domain and is not external."""
        parsed_url = urlparse(url)
        return parsed_url.netloc == urlparse(self.start_url).netloc and parsed_url.scheme in ["http", "https"]

    async def crawl(self):
        """Run async crawler"""
        start_time = time.time()  # Track start time

        async with aiohttp.ClientSession() as session:
            while self.to_visit:
                tasks = []
                # Process up to 10 pages in parallel
                for _ in range(min(len(self.to_visit), 10)):
                    url = self.to_visit.pop(0)
                    tasks.append(self.scrape_page(session, url))
                await asyncio.gather(*tasks)

        end_time = time.time()  # Track end time
        total_time = round(end_time - start_time, 2)
        print(f"\n✅ Scraping completed in {total_time} seconds.\n")

        return pd.DataFrame(self.data)

In [4]:
# Usage:
start_url = "https://peakscurative.com/"  # YOUR WEBSITE HERE
scraper = AsyncWebScraper(start_url)

# Run the scraper (in a Jupyter Notebook use "await")
df = await scraper.crawl()
display(df)

Scraping: https://peakscurative.com/
Scraping: https://peakscurative.com
Scraping: https://peakscurative.com/mens-sexual-health/
Scraping: https://peakscurative.com/mens-sexual-health-peaknow-melts/
Scraping: https://peakscurative.com/nasal-sprays/
Scraping: https://peakscurative.com/mens-sexual-health-peaknow-gummies/
Scraping: https://peakscurative.com/product/ignite15g/
Scraping: https://peakscurative.com/mens-health-viagra-cialis/
Scraping: https://peakscurative.com/women-sexual-health/
Scraping: https://peakscurative.com/product/arousal-melts-x10-units/
Scraping: https://peakscurative.com/scream-cream/
Scraping: https://peakscurative.com/weight-loss-solutions-you-are-ready-for-change/
Scraping: https://peakscurative.com/hair-growth
Scraping: https://peakscurative.com/skin-care-solutions/
Scraping: https://peakscurative.com/product/vitality-boost/
Scraping: https://peakscurative.com/product/procoll/
Scraping: https://peakscurative.com/peak-bundle-packages/
Scraping: https://peakscu

Scraping: https://peakscurative.com?add-to-cart=16493&quantity=1&e-redirect=https://peakscurative.com/product/arousal-bundle-for-him-her/
Scraping: https://peakscurative.com/product/ultimate-mane-magic-bundle/?variant=16688&utm_medium=cpc&utm_source=bing&utm_campaign=Bing+Shopping&add-to-cart=16688
Scraping: https://peakscurative.com/product/arousal-bundle-for-him-her/?add-to-cart=13617
Scraping: https://peakscurative.com?add-to-cart=16667&quantity=1&e-redirect=https://peakscurative.com/product/pleasure-bundle-for-her/
Scraping: https://peakscurative.com/product/pleasure-bundle-for-her/?add-to-cart=13617
Scraping: https://peakscurative.com?add-to-cart=18281&quantity=1&e-redirect=https://peakscurative.com/product/excitement-bundle-for-him-her/
Failed to scrape https://peakscurative.com?add-to-cart=16493&quantity=1&e-redirect=https://peakscurative.com/product/arousal-bundle-for-him-her/: 
Failed to scrape https://peakscurative.com?add-to-cart=18281&quantity=1&e-redirect=https://peakscura

,url,text
0,https://peakscurative.com/,"Peaks\nCurative - For\nWeight\nLoss, Sexual\nH..."
1,https://peakscurative.com,"Peaks\nCurative - For\nWeight\nLoss, Sexual\nH..."
2,https://peakscurative.com/mens-sexual-health-p...,Men's\nSexual\nHealth - Peak\nNow\nMelts\nFor\...
3,https://peakscurative.com/product/ignite15g/,Ignite - Better\nExperience - Peak\nNow\nSexua...
4,https://peakscurative.com/product/arousal-melt...,Arousal\nMelts\nFor\nHer - Sublingual\nMelts\n...
...,...,...
132,https://peakscurative.com/wp-login.php?redirec...,Log\nIn ‹ Peaks\nCurative — WordPress\nLog\nIn...
133,https://peakscurative.com/my-peak-cart/,My\nPeak\nCart - Peaks\nCurative\nSexual\nHeal...
134,https://peakscurative.com/wp-login.php,Log\nIn ‹ Peaks\nCurative — WordPress\nLog\nIn...
135,https://peakscurative.com/my-peak-cart/?add-to...,My\nPeak\nCart - Peaks\nCurative\nSexual\nHeal...


In [5]:
# Save functions with fixed encoding

def save_to_html(df, filename="website_text.html"):
    """Save extracted website text as a structured HTML file."""
    with open(filename, "w", encoding="utf-8") as file:
        file.write("<html><head><title>Extracted Website Text</title></head><body>\n")
        file.write("<h1>Extracted Website Content</h1>\n")
        
        for index, row in df.iterrows():
            formatted_text = row['text'].replace("\n", "<br>")
            file.write(f"<h2><a href='{row['url']}'>{row['url']}</a></h2>\n")
            file.write("<hr>\n")
            file.write(f"<p>{formatted_text}</p>\n")
            file.write("<br><hr><br>\n")
        
        file.write("</body></html>")
    print(f"✅ Extracted text saved as {filename} (HTML Format)")

save_to_html(df, "website_text.html")

✅ Extracted text saved as website_text.html (HTML Format)


In [6]:
def save_to_text(df, filename="website_text.txt"):
    """Save extracted website text in a structured TXT file."""
    with open(filename, "w", encoding="utf-8") as file:
        for index, row in df.iterrows():
            file.write(f"URL: {row['url']}\n")
            file.write("="*80 + "\n")
            file.write(row["text"] + "\n\n")
            file.write("-"*80 + "\n\n")
    print(f"✅ Extracted text saved as {filename} (Readable TXT Format)")

save_to_text(df, "website_text.txt")

✅ Extracted text saved as website_text.txt (Readable TXT Format)
